In [1]:
import duckdb

con=duckdb.execute("""IMPORT DATABASE '../data/db';""")
df = con.execute(""" SELECT
      c.crew_id,
      ARRAY_AGG(
          STRUCT_PACK(
              ship := s.ship_id,
              start_ts := c.start_ts_utc,
              end_ts := c.end_ts_utc,
              role := c.role
          )
          ORDER BY c.start_ts_utc
      ) AS shifts
  FROM crew_shifts_hist c
  JOIN sailings s ON c.sailing_id = s.sailing_id
  GROUP BY c.crew_id
  ORDER BY c.crew_id;""").df()
con.close()
df

,crew_id,shifts
0,C001,"[{'ship': 'S1', 'start_ts': 2025-05-04 18:00:0..."
1,C002,"[{'ship': 'S1', 'start_ts': 2025-05-02 02:00:0..."
2,C003,"[{'ship': 'S1', 'start_ts': 2025-05-04 18:00:0..."
3,C005,"[{'ship': 'S2', 'start_ts': 2025-05-02 09:00:0..."
4,C006,"[{'ship': 'S2', 'start_ts': 2025-05-02 10:00:0..."
...,...,...
76,C101,"[{'ship': 'S2', 'start_ts': 2025-05-01 18:00:0..."
77,C103,"[{'ship': 'S2', 'start_ts': 2025-05-05 02:00:0..."
78,C105,"[{'ship': 'S1', 'start_ts': 2025-05-06 18:00:0..."
79,C107,"[{'ship': 'S2', 'start_ts': 2025-05-07 10:00:0..."


In [2]:
import plotly.graph_objects as go
import pandas as pd
import plotly.graph_objects as go
import pandas as pd

def plot_crew_schedule(df, crew_id, plot=False):
    """
    Plots a timeline of shifts for a single crew member using Plotly.

    Parameters
    ----------
    df : pandas.DataFrame
        Must have columns ['crew_id', 'shifts'] where 'shifts' is a collection of
        tuples (ship_id, start_ts, end_ts, role)
    crew_id : str
        Crew member ID to filter on
    """
    # Filter by crew member
    dff = df[df['crew_id'] == crew_id].copy()
    if dff.empty:
        print(f"No data found for crew_id: {crew_id}")
        return

    # Extract and deduplicate shifts
    shifts = {(d["ship"], d["start_ts"], d['end_ts'], d["role"]) for d in dff["shifts"].explode()}
    if not shifts:
        print(f"No shifts found for crew_id: {crew_id}")
        return

    # Ensure all shifts have the same role
    roles = {x[3] for x in shifts}
    assert len(roles) == 1, f"Multiple roles found for {crew_id}: {roles}"

    ordered_shifts = sorted(shifts, key=lambda x: x[1])
    ships = sorted({s[0] for s in ordered_shifts})

    # Map ships to numeric y positions for plotting
    ship_to_y = {ship: i for i, ship in enumerate(ships)}

    fig = go.Figure()

    for ship, start, end, role in ordered_shifts:
        shift_length = end - start
        hours = round(shift_length.total_seconds() / 3600, 1)
        legend_label = f"{ship} ({hours}h)"

        fig.add_trace(go.Scatter(
            x=[start, end],
            y=[ship_to_y[ship]] * 2,
            mode='lines',
            line=dict(width=10),
            name=legend_label,
            hovertemplate=(
                f"Ship: {ship}<br>"
                f"Role: {role}<br>"
                f"Shift length: {hours} hours<br>"
                "%{x|%Y-%m-%d %H:%M}"
            )
        ))

    fig.update_layout(
        title=f"Crew Schedule for {crew_id} (Role: {list(roles)[0]})",
        xaxis=dict(title="Time"),
        yaxis=dict(
            title="Ship",
            tickmode="array",
            tickvals=list(ship_to_y.values()),
            ticktext=list(ship_to_y.keys())
        ),
        showlegend=True,
        height=300 + 50 * len(ships)
    )
    if plot:
        fig.show()
    return ordered_shifts

ordered_shifts_dict = {
    code: plot_crew_schedule(df, code) for code in set(df["crew_id"])
}


In [8]:
from datetime import datetime


def _hours_between(a, b):
    try:
        t1 = pd.to_datetime(a)
        t2 = pd.to_datetime(b)
        return (t2 - t1).total_seconds() / 3600
    except Exception:
        return None


rests = {
    key: [
        h
        for i in range(len(values) - 1)
        if (h := _hours_between(values[i][2],values[i + 1][1])) is not None
    ]
    for key, values in ordered_shifts_dict.items()
}


In [9]:
ordered_shifts_dict["C001"]

[('S1',
  datetime.datetime(2025, 5, 4, 18, 0, tzinfo=<DstTzInfo 'America/Bogota' -05-1 day, 19:00:00 STD>),
  datetime.datetime(2025, 5, 5, 2, 0, tzinfo=<DstTzInfo 'America/Bogota' -05-1 day, 19:00:00 STD>),
  'security'),
 ('S1',
  datetime.datetime(2025, 5, 8, 10, 0, tzinfo=<DstTzInfo 'America/Bogota' -05-1 day, 19:00:00 STD>),
  datetime.datetime(2025, 5, 8, 18, 0, tzinfo=<DstTzInfo 'America/Bogota' -05-1 day, 19:00:00 STD>),
  'security'),
 ('S3',
  datetime.datetime(2025, 5, 9, 2, 0, tzinfo=<DstTzInfo 'America/Bogota' -05-1 day, 19:00:00 STD>),
  datetime.datetime(2025, 5, 9, 10, 0, tzinfo=<DstTzInfo 'America/Bogota' -05-1 day, 19:00:00 STD>),
  'security'),
 ('S1',
  datetime.datetime(2025, 5, 12, 10, 0, tzinfo=<DstTzInfo 'America/Bogota' -05-1 day, 19:00:00 STD>),
  datetime.datetime(2025, 5, 12, 18, 0, tzinfo=<DstTzInfo 'America/Bogota' -05-1 day, 19:00:00 STD>),
  'security'),
 ('S1',
  datetime.datetime(2025, 5, 14, 10, 0, tzinfo=<DstTzInfo 'America/Bogota' -05-1 day, 19:00:

In [10]:
rests["C001"]

[80.0,
 8.0,
 72.0,
 40.0,
 64.0,
 48.0,
 88.0,
 8.0,
 0.0,
 32.0,
 8.0,
 32.0,
 16.0,
 216.0,
 16.0,
 40.0,
 0.0,
 24.0,
 112.0,
 32.0,
 0.0,
 24.0,
 8.0,
 32.0,
 80.0,
 16.0,
 88.0,
 32.0,
 0.0,
 32.0,
 16.0,
 96.0,
 8.0,
 24.0,
 24.0,
 8.0,
 24.0,
 24.0,
 80.0,
 16.0,
 16.0,
 24.0]

In [12]:
ordered_shifts_dict['C064']

[('S1',
  datetime.datetime(2025, 5, 1, 18, 0, tzinfo=<DstTzInfo 'America/Bogota' -05-1 day, 19:00:00 STD>),
  datetime.datetime(2025, 5, 2, 2, 0, tzinfo=<DstTzInfo 'America/Bogota' -05-1 day, 19:00:00 STD>),
  'cook'),
 ('S3',
  datetime.datetime(2025, 5, 2, 2, 0, tzinfo=<DstTzInfo 'America/Bogota' -05-1 day, 19:00:00 STD>),
  datetime.datetime(2025, 5, 2, 10, 0, tzinfo=<DstTzInfo 'America/Bogota' -05-1 day, 19:00:00 STD>),
  'cook'),
 ('S2',
  datetime.datetime(2025, 5, 6, 10, 0, tzinfo=<DstTzInfo 'America/Bogota' -05-1 day, 19:00:00 STD>),
  datetime.datetime(2025, 5, 6, 18, 0, tzinfo=<DstTzInfo 'America/Bogota' -05-1 day, 19:00:00 STD>),
  'cook'),
 ('S2',
  datetime.datetime(2025, 5, 7, 18, 0, tzinfo=<DstTzInfo 'America/Bogota' -05-1 day, 19:00:00 STD>),
  datetime.datetime(2025, 5, 8, 2, 0, tzinfo=<DstTzInfo 'America/Bogota' -05-1 day, 19:00:00 STD>),
  'cook'),
 ('S2',
  datetime.datetime(2025, 5, 9, 10, 0, tzinfo=<DstTzInfo 'America/Bogota' -05-1 day, 19:00:00 STD>),
  datetime

In [13]:
rests['C064']

[0.0,
 96.0,
 24.0,
 32.0,
 16.0,
 -8.0,
 0.0,
 8.0,
 -8.0,
 24.0,
 0.0,
 0.0,
 88.0,
 24.0,
 56.0,
 104.0,
 24.0,
 104.0,
 120.0,
 80.0,
 56.0,
 0.0,
 0.0,
 8.0,
 16.0,
 8.0,
 -8.0,
 56.0,
 48.0,
 24.0,
 24.0,
 48.0,
 112.0,
 56.0,
 8.0,
 16.0,
 8.0,
 16.0,
 48.0,
 -8.0,
 56.0,
 24.0,
 24.0,
 16.0,
 112.0,
 0.0,
 24.0,
 16.0,
 32.0]

In [15]:
_= plot_crew_schedule(df, 'C064', plot=True)

In [16]:
lengths = {
    crew_id: [(end - start).total_seconds() / 3600 for (ship, start, end, role) in ordered_shifts or []]
    for crew_id, ordered_shifts in ordered_shifts_dict.items()
}


In [18]:
# Build summary table with additional rest metrics: rests == 0 and rests < 0
data = []
for crew_id, shift_list in ordered_shifts_dict.items():
    shift_list = shift_list or []
    shift_lengths = lengths.get(crew_id, []) or []
    rest_list = rests.get(crew_id, []) or []

    data.append({
        "crew_id": crew_id,
        "num_shifts": len(shift_list),
        "num_shifts_gt_8h": sum(1 for h in shift_lengths if h > 8),
        "num_rests": len(rest_list),
        "num_rests_lt_12h": sum(1 for r in rest_list if r < 12),
        "num_rests_eq_0h": sum(1 for r in rest_list if r == 0),
        "num_rests_lt_0h": sum(1 for r in rest_list if r < 0),
    })

report_df = pd.DataFrame(data).sort_values("crew_id").reset_index(drop=True)
report_df


,crew_id,num_shifts,num_shifts_gt_8h,num_rests,num_rests_lt_12h,num_rests_eq_0h,num_rests_lt_0h
0,C001,43,0,42,10,4,0
1,C002,44,0,43,14,5,3
2,C003,49,0,48,14,3,7
3,C005,72,0,71,31,13,7
4,C006,53,0,52,19,13,4
...,...,...,...,...,...,...,...
76,C101,47,0,46,15,6,4
77,C103,44,0,43,16,8,4
78,C105,52,0,51,16,8,2
79,C107,50,0,49,18,9,2


The following metrics correspond to 3 KPI's that measure the quality of the scheduling presented in the crew_shifts_hist.csv

In [22]:
consecutive_shifts = sum(report_df["num_rests_eq_0h"])
print("There are", consecutive_shifts, "consecutive shifts.")
infeasible_assignations = sum(report_df["num_rests_lt_0h"])
print("There are", infeasible_assignations, "infeasible assignations.")
total_not_allowed_rests = sum(report_df["num_rests_lt_12h"])
print("There are", total_not_allowed_rests, "rests that are shorter than 12 hours.")
total_number_of_shifts = sum(report_df["num_shifts"])
print("There are", total_number_of_shifts, "shifts in total.")

There are 478 consecutive shifts.
There are 237 infeasible assignations.
There are 1101 rests that are shorter than 12 hours.
There are 3686 shifts in total.


## Consecutive Shifts
This shifts are that were concatenated with 0 rest time, this means the crew member is supposed to work 16 hours in a row.

## Infeasible Assignations
There are cases where the crew member is supposed to work in 2 Ships during the same hours.

The C064 has examples of both

In [23]:
_=plot_crew_schedule(df,"C064", plot=True)